In [44]:
import pandas as pd
import numpy as np
import progressbar
import matplotlib.pyplot as plt
from IMPORT_DATAFRAME_JSON_HDF5 import *
%matplotlib notebook
import os


In [106]:
Phase_2 = pd.read_csv('power_today_minute_2.csv', names = ['timestamp','P'])
Phase_2_mit_delta = Phase_2
Phase_2_mit_delta['P_delta'] = 0
Event_df = pd.DataFrame(columns = ['timestamp','minuten_index' ,'Ladevorgang', 'Status', 'Ladeleistung'])

for row in range(1,len(Phase_2)):
    # daten import
    Timestamp = Phase_2.iloc[row]['timestamp']
    minuten_index = row
    P_new = Phase_2.iloc[row]['P']
    P_old = Phase_2.iloc[row-1]['P']
    P_delta = P_new-P_old
    Phase_2_mit_delta.loc[row, 'P_delta'] = P_delta
    
    # ist delta P relevant
    if(abs(P_delta) > 50.0):
        # Prüfen ob Ladenvorgang beginnt/beendet
        if(P_delta > 100.0):
            Event_df.loc[len(Event_df)]= [Timestamp, minuten_index, 'ein', 'not checked', 'nan']
            #Event_df.append(df, ignore_index=True)
        else:
            None
        
    #Check events
    for row_event in range(0,len(Event_df)):
        #condition to check event
        minuten_index_event = Event_df.iloc[row_event]['minuten_index'] 
        if (minuten_index_event+3 == minuten_index):
            Ladeleistung = Phase_2.iloc[minuten_index_event+3]['P'] - Phase_2.iloc[minuten_index_event-3]['P'] 
            sum_delta_P = Phase_2_mit_delta[minuten_index_event:minuten_index_event+3]['P_delta'].sum()
            deviation = abs(Ladeleistung-sum_delta_P)/Ladeleistung 
            if abs(Ladeleistung-sum_delta_P)/Ladeleistung >0.01:
                print(deviation)
                Event_df = Event_df.drop(row_event)
            else:
                Event_df.loc[row_event, ['Status', 'Ladeleistung']] = ['checked', Ladeleistung]

0.566908563135
0.664326573063
0.297942386831


In [107]:
Event_df

,timestamp,minuten_index,Ladevorgang,Status,Ladeleistung
0,1.505279e+09,430,ein,checked,3442
1,1.505287e+09,557,ein,checked,3846
2,1.505291e+09,628,ein,checked,3642


In [84]:
Phase_2_mit_delta.to_csv('test.csv')